<a href="https://colab.research.google.com/github/PhilippMatthes/diplom/blob/master/src/shl-deep-learning-timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a deep CNN to directly classify SHL timeseries data

In [1]:
# Get needed auxiliary files for colab
!git clone https://github.com/philippmatthes/diplom
%cd /content/diplom/src
!mkdir shl-dataset
!wget -nc -O shl-dataset/challenge-2019-train_torso.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
!wget -nc -O shl-dataset/challenge-2019-train_bag.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
!wget -nc -O shl-dataset/challenge-2019-train_hips.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_hips.zip
!wget -nc -O shl-dataset/challenge-2020-train_hand.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2020/challenge-2020-train_hand.zip

fatal: destination path 'diplom' already exists and is not an empty directory.
/content/diplom/src
mkdir: cannot create directory ‘shl-dataset’: File exists
--2021-08-05 08:12:07--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl-dataset.org)|37.187.125.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852446972 (5.5G) [application/zip]
Saving to: ‘shl-dataset/challenge-2019-train_torso.zip’

shl-dataset/challen 100%[===================>]   5.45G  3.39MB/s    in 20m 30s 

2021-08-05 08:32:38 (4.54 MB/s) - ‘shl-dataset/challenge-2019-train_torso.zip’ saved [5852446972/5852446972]

--2021-08-05 08:32:38--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl

In [2]:
# Switch to src dir and select tensorflow
%cd /content/diplom/src
%tensorflow_version 1.x

/content/diplom/src
TensorFlow 1.x selected.


In [3]:
from pathlib import Path

MODEL_DIR = Path('models/shl-model-sparse-cat-crossentropy-resnet.h5')

In [4]:
# Define all datasets to train our model on

DATASET_DIRS = [
    Path('shl-dataset/challenge-2019-train_torso.zip'),
    Path('shl-dataset/challenge-2019-train_bag.zip'),
    Path('shl-dataset/challenge-2019-train_hips.zip'),
    Path('shl-dataset/challenge-2020-train_hand.zip'),
]

In [5]:
# Create scalers that we will fit on our data

from sklearn.preprocessing import PowerTransformer

acc_scaler = PowerTransformer()
mag_scaler = PowerTransformer()
gyr_scaler = PowerTransformer()

In [6]:
from tensorflow import keras

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5367602516450504716, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2037885184663652605
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17378095255737902776
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15964005991
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8607148834644542387
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [7]:
# Create our model

from tools.architectures import make_resnet
from tools.dataset import shl_dataset_label_order

if MODEL_DIR.is_file():
    # Load model for transfer learning
    model = keras.models.load_model(MODEL_DIR)
else:
    # Create model
    model = make_resnet(
        input_shape=[500, 3], # Magnitudes of ACC, MAG and Gyr, 500 data points each 
        output_classes=len(shl_dataset_label_order)
    )
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['acc']
    )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
import numpy as np

from tools.dataset import load_zipped_shl_dataset
from tools.export import export_power_transformer

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

for dataset_dir in DATASET_DIRS:
    # Load dataset from zip file into temporary directory
    dataset = load_zipped_shl_dataset(dataset_dir, tqdm=tqdm)

    # Fit and export scalers
    acc_mag_scaled = acc_scaler.fit_transform(dataset.acc_mag)
    mag_mag_scaled = mag_scaler.fit_transform(dataset.mag_mag)
    gyr_mag_scaled = gyr_scaler.fit_transform(dataset.gyr_mag)
    export_power_transformer(acc_scaler, 'models/acc-transformer.json')
    export_power_transformer(mag_scaler, 'models/mag-transformer.json')
    export_power_transformer(gyr_scaler, 'models/gyr-transformer.json')

    # Prepare our training data
    X = np.stack([
        acc_mag_scaled,
        mag_mag_scaled, 
        gyr_mag_scaled,
    ], axis=2)
    y = dataset.labels

    # Save some memory
    del dataset
    del acc_mag_scaled
    del mag_mag_scaled
    del gyr_mag_scaled

    # Compute class weights for unbiased training
    class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    class_weights = dict(zip(np.unique(y), class_weights)) # Keras adaption

    # Train model
    callbacks = [
        keras.callbacks.CSVLogger(f'{MODEL_DIR}.log', append=MODEL_DIR.is_file()),
        keras.callbacks.ModelCheckpoint(
            str(MODEL_DIR), save_best_only=True, monitor='val_loss'
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001
        ),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ]
    model.fit(
        X, 
        y,
        epochs=10, 
        batch_size=32,
        class_weight=class_weights, 
        callbacks=callbacks,
        validation_split=0.1,
        verbose=1,
    )

    # Save some memory
    del X
    del y
    

Extracting shl-dataset/challenge-2019-train_torso.zip: 100%|██████████| 22/22 [03:45<00:00, 10.27s/it]


Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Acc_mag Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Mag_mag Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Gyr_mag Import Done
Labels Import Done
Train on 176464 samples, validate on 19608 samples
Epoch 1/10
176464/176464 [==============================] - 288s 2ms/sample - loss: 0.6739 - acc: 0.7000 - val_loss: 0.5270 - val_acc: 0.7933
Epoch 2/10
176464/176464 [==============================] - 230s 1ms/sample - loss: 0.4823 - acc: 0.7890 - val_loss: 0.4281 - val_acc: 0.8283
Epoch 3/10
176464/176464 [==============================] - 161s 914us/sample - loss: 0.4157 - acc: 0.8243 - val_loss: 0.4337 - val_acc: 0.8340
Epoch 4/10
136800/176464 [======================>.......] - ETA: 35s - loss: 0.3819 - acc: 0.8404